<center style="font-size:48px;">Modeling</center>

<center>My attempt at utilizing machine learning to identify some initial data ppints to further invesigate</center>

# Import Libraries and Data

## Libraries

In [1]:
# Data Science
import pandas as pd
import joblib

# Machine Learning
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

## Data

In [2]:
# Location
locations1 = pd.read_csv('../CheckPoints/Locations1_Clean.csv', parse_dates=['Timestamp'], index_col=0)
locations2 = pd.read_csv('../CheckPoints/Locations2_Clean.csv', parse_dates=['Timestamp'], index_col=0)
locations = pd.concat([locations1, locations2])
# Purchase Data
buys = pd.read_csv('../CheckPoints/Buys_Clean.csv', parse_dates=['timestamp'], index_col=0)

# Machine Learning

## Location

Create a preprocessing pipeline. We need to drop the timestamp column as we already have the datetime parts seperated into their own variable. We can also drop the CarID variable as there is a one to one relationship with the CarID. We will need to encode the names variables as well as the job title and type.

In [3]:
locPreprocessor = ColumnTransformer(
    [
        ( 'dropper', 'drop', ['Timestamp', 'id'] ),
        ('encoder', OneHotEncoder(), ['LastName', 'FirstName', 'CurrentEmploymentType', 'CurrentEmploymentTitle'])
    ],
    remainder='passthrough'
)

We will use two algorithms that are implmented in scikit-learn; IsolationForest and LocalOutlierFactor. First lets look at IsolationForest.

In [4]:
# Model Pipeline
locIso = Pipeline(
    [
        ('preprocessor', locPreprocessor),
        ('model', IsolationForest())
    ]
)
# Get Predictions
predslocIso = locIso.fit_predict(locations)
pd.DataFrame(predslocIso)[0].unique()

array([1])

It looks like this algorithm didnt find any outliers.

Now lets try LocalOutlierFactor.

In [5]:
# Model Pipeline
locLocalpipe = Pipeline(
    [
        ('preprocessor', locPreprocessor),
        ('model', LocalOutlierFactor(n_jobs = -1))
    ]
)
# Get Predictions
predsLocLocal = locLocalpipe.fit_predict(locations)
pd.DataFrame(predsLocLocal)[0].unique()

array([ 1, -1])

Lets look at which outliers have been identified; the points labeld -1.

In [27]:
predsLocLocalClean = [ True if x == -1 else False for x in predsLocLocal]
locations[predsLocLocalClean]

,Timestamp,id,lat,long,LastName,FirstName,CurrentEmploymentType,CurrentEmploymentTitle,day,hour,minute,second,Weekend
309322,2014-01-12 00:00:03,29,36.061040,24.892037,Ovan,Bertrand,Facilities,Facilities Group Manager,12,0,0,3,True
309323,2014-01-12 00:00:06,29,36.060997,24.892243,Ovan,Bertrand,Facilities,Facilities Group Manager,12,0,0,6,True
309324,2014-01-12 00:00:07,29,36.060941,24.892200,Ovan,Bertrand,Facilities,Facilities Group Manager,12,0,0,7,True
309325,2014-01-12 00:00:08,29,36.060844,24.892220,Ovan,Bertrand,Facilities,Facilities Group Manager,12,0,0,8,True
309326,2014-01-12 00:00:10,29,36.060662,24.892193,Ovan,Bertrand,Facilities,Facilities Group Manager,12,0,0,10,True
309327,2014-01-12 00:00:16,29,36.060268,24.892338,Ovan,Bertrand,Facilities,Facilities Group Manager,12,0,0,16,True
309328,2014-01-12 00:00:18,29,36.060154,24.892508,Ovan,Bertrand,Facilities,Facilities Group Manager,12,0,0,18,True
309329,2014-01-12 00:00:20,29,36.060152,24.892640,Ovan,Bertrand,Facilities,Facilities Group Manager,12,0,0,20,True
309330,2014-01-12 00:00:22,29,36.060019,24.892860,Ovan,Bertrand,Facilities,Facilities Group Manager,12,0,0,22,True
309334,2014-01-12 00:00:29,29,36.059829,24.893377,Ovan,Bertrand,Facilities,Facilities Group Manager,12,0,0,29,True


## Purchases

Create a preprocessing pipeline. We need to drop the timestamp column as we already have the datetime parts seperated into their own variable. We can also drop the CarID variable as there is a one to one relationship with the CarID. Lets also drop the minute and seconda columns. With a chunk of the purchases not haveing this because they were batch reported it could alter results. We will need to encode the names, locations,  job title and type variables.

In [6]:
Buypreprocessor = ColumnTransformer(
    [
        ( 'dropper', 'drop', ['timestamp',  'CarID', 'minute', 'second'] ),
        ('encoder', OneHotEncoder(), ['location', 'LastName', 'FirstName', 'CurrentEmploymentType', 'CurrentEmploymentTitle'])
    ],
    remainder='passthrough'
)

Similar to above we will use two algorithms that are implmented in scikit-learn; IsolationForest and LocalOutlierFactor. First lets look at IsolationForest.

In [7]:
# Model Pipeline
BuyIso = Pipeline(
    [
        ('preprocessor', Buypreprocessor),
        ('model', IsolationForest())
    ]
)
# Get Predictions
predsBuyIso = BuyIso.fit_predict(buys)
pd.DataFrame(predsBuyIso)[0].unique()

array([1])

Mhmmm, once again no outliers for this algorithm.

In [25]:
# Model Pipeline
BuyLocal = Pipeline(
    [
        ('preprocessor', Buypreprocessor),
        ('model', LocalOutlierFactor(n_jobs = -1))
    ]
)
# Get Predictions
predsBuyLocal = BuyLocal.fit_predict(buys)
pd.DataFrame(predsBuyLocal)[0].unique()

array([ 1, -1])

Lets look at which outliers have been identified; the points labeld -1.

In [26]:
predsBuyLocal = [ True if x == -1 else False for x in predsBuyLocal]
buys[predsBuyLocal]

,timestamp,location,price,FirstName,LastName,Is_Loyalty,day,hour,minute,second,Weekend,CarID,CurrentEmploymentType,CurrentEmploymentTitle
92,2014-01-06 15:39:00,Abila Airport,612.47,Henk,Mies,True,6,15,39,0,False,100.0,Facilities,Truck Driver
414,2014-01-09 09:54:00,Carlyle Chemical Inc.,529.47,Valeria,Morlun,True,9,9,54,0,False,100.0,Facilities,Truck Driver
479,2014-01-09 14:26:00,Carlyle Chemical Inc.,382.43,Valeria,Morlun,True,9,14,26,0,False,100.0,Facilities,Truck Driver
480,2014-01-09 14:28:00,Kronos Pipe and Irrigation,423.14,Irene,Nant,True,9,14,28,0,False,100.0,Facilities,Truck Driver
842,2014-01-13 19:20:00,Frydos Autosupply n' More,10000.00,Lucas,Alcazar,False,13,19,20,0,False,1.0,Information Technology,IT Helpdesk
898,2014-01-14 08:12:00,Hallowed Grounds,39.52,Kare,Orilla,False,14,8,12,0,False,27.0,Engineering,Drill Technician
906,2014-01-14 11:01:00,Stewart and Sons Fabrication,540.19,Benito,Hawelon,True,14,11,1,0,False,100.0,Facilities,Truck Driver
907,2014-01-14 11:45:00,Nationwide Refinery,570.95,Valeria,Morlun,True,14,11,45,0,False,100.0,Facilities,Truck Driver
1158,2014-01-16 09:52:00,Carlyle Chemical Inc.,419.93,Valeria,Morlun,True,16,9,52,0,False,100.0,Facilities,Truck Driver
1372,2014-01-18 12:03:00,Chostus Hotel,600.00,Sten,Sanjorge Jr.,True,18,12,3,0,True,31.0,Executive,President/CEO


Save our two models.

In [38]:
joblib.dump(locLocalpipe, '../models/LocationLocalOutlierFactor.sav')
joblib.dump(BuyLocal, '../models/PurchaseLocalOutlierFactor.sav')

['../models/PurchaseLocalOutlierFactor.sav']

<div>
    <span  style="width:600px;display:inline-block;text-align:left">
        <a href="./CleanUp.ipynb">&#60;&#60;Data Cleanup</a>
    </span>
    <span style="width:600px;display:inline-block;text-align:right">
        <a href="./ObservationsQ1.ipynb">Observations Question 1&#62;&#62;</a>
    </span>
</div>
<div>
    <center>
        <span style="width:200px;display:inline-block;text-align:center">
            <a href="./Master.ipynb">Master Notebook</a>
        </span>
        <span style="width:200px;display:inline-block;text-align:center">
            <a href="../README.md">Table of Contents</a>
        </span>
    </center>
</div>